In [1]:
import time

# Total duration in seconds
total_duration = 1000  #
# Number of iterations (you can change this)
iterations = total_duration
# Delay per iteration
delay = total_duration / iterations

for i in range(iterations):
    # Do something here if needed
    print(f"Step {i+1}/{iterations}")
    time.sleep(delay)

print("Done.")

Step 1/1000
Step 2/1000
Step 3/1000
Step 4/1000
Step 5/1000
Step 6/1000
Step 7/1000
Step 8/1000
Step 9/1000
Step 10/1000
Step 11/1000
Step 12/1000
Step 13/1000
Step 14/1000
Step 15/1000
Step 16/1000
Step 17/1000
Step 18/1000
Step 19/1000
Step 20/1000
Step 21/1000
Step 22/1000
Step 23/1000
Step 24/1000
Step 25/1000
Step 26/1000
Step 27/1000
Step 28/1000
Step 29/1000
Step 30/1000
Step 31/1000
Step 32/1000
Step 33/1000
Step 34/1000
Step 35/1000
Step 36/1000
Step 37/1000
Step 38/1000
Step 39/1000
Step 40/1000
Step 41/1000
Step 42/1000
Step 43/1000
Step 44/1000
Step 45/1000
Step 46/1000
Step 47/1000
Step 48/1000
Step 49/1000
Step 50/1000
Step 51/1000
Step 52/1000
Step 53/1000
Step 54/1000
Step 55/1000
Step 56/1000
Step 57/1000
Step 58/1000
Step 59/1000
Step 60/1000
Step 61/1000
Step 62/1000
Step 63/1000
Step 64/1000
Step 65/1000
Step 66/1000
Step 67/1000
Step 68/1000
Step 69/1000
Step 70/1000
Step 71/1000
Step 72/1000
Step 73/1000
Step 74/1000
Step 75/1000
Step 76/1000
Step 77/1000
Step 78/

In [2]:
%run ./"0 Imports and functions.ipynb"

/home/ben/miniconda3/bin/python
3.8.5 (default, Sep  4 2020, 07:30:14) 
[GCC 7.3.0]
sys.version_info(major=3, minor=8, micro=5, releaselevel='final', serial=0)
astropy 5.1.dev153+gb740594dc
spectral_cube 0.6.1.dev22+g003ef16
spectral_cube file path /home/ben/.local/lib/python3.8/site-packages/spectral_cube/__init__.py
reproject 0.8
3.3.4 Matplotlib
/home/ben/miniconda3/lib/python3.8/site-packages/matplotlib/__init__.py
1.23.1 Numpy
astrodendro_file: /home/ben/.local/lib/python3.8/site-packages/astrodendro/__init__.py
Results will be saved to Directory ./Result Files
Cubes will be saved and loaded with Directory ./Spectral Cubes
Plots will be saved to Directory ./Plots
Dendrograms will be saved to Directory ./Dendrograms


In [3]:
# Pointing_Information

Pointing_Information = {}

#This is the stuff that needs changing between cubes

Pointing_Information["Original_File_Name"] = "hcn4_3_KT18_Tmb.fits"
File_Descriptor='cmz_HCN_J4_3_'

Pointing_Information["File_Descriptor"] = File_Descriptor
Pointing_Information["target"] = "CMZ"#
Pointing_Information["center"] = SkyCoord('-00d03m20.76s  ', '-00d02m46.176s', frame='galactic') #the center of the cmz
Pointing_Information["crop_center"] = SkyCoord('00d00m00.00s  ', '-00d00m00.000s', frame='galactic') #the center of the image, used for cropping

desired_beam_size = 3*u.pc #I add this to the PI later. choose this based on the largest common beam size between the compared observations
distance = 8.178*10**3*u.pc
Pointing_Information["distance"] = distance.to(u.Mpc) #in Mpc
Pointing_Information["target_image_rotation"]=0*u.deg #this is the rotation of the specific image, not the target. (use clockwise rotation angle)
Pointing_Information["target_inclination"]=0*u.deg
Pointing_Information["target_velocity"]=0*u.km/u.s #the speed NGC253 is moving away from us
Pointing_Information["vaxis"]=0 #which axis is the velocity
Pointing_Information["desired_velocity_resolution"] = 2.5*u.km/u.s
ovs = 5 #how much do you desire to oversample the beam by
desired_FOV = [140,700]*u.pc# pc normal to the disk, across the disk
Pointing_Information["Corresponding_Continuum"] = 'gc_850micron_dust.fits' # this is for the band 7, not for HCN J1-0 #The continuum image for this band
#                                                 '4.3pc_beam_CMZ_850um_Cont_140x800pc.fits' # this is the JCMT continuum for 850microns (the CO 3-2 and HCN, HCO 4-3)


print("done")

gc.collect








#This stuff will input automagically if the rest is correct

sc = SpectralCube.read("Spectral Cubes/"+Pointing_Information["Original_File_Name"])
header = sc.header
print()
try:
    freq = header["RESTFREQ"]*u.Hz# assuming the header is in Hz
    Pointing_Information['wavelength']=299792458*u.m/header["RESTFREQ"]#
    Pointing_Information['restfreq']=freq         
except:
    freq = header["RESTFRQ"]*u.Hz#
    Pointing_Information['wavelength']=299792458*u.m/header["RESTFRQ"]#            
    Pointing_Information['restfreq']=freq 

######calculate teh beam size of the original image:

if (header['CUNIT1'].find("deg")!=-1):
    CUNIT = 1*u.degree
    Pointing_Information["CUNIT"]=CUNIT
else:
    print("The header should show CUNIT in degrees. If not, just fix this or write CUNIT = the unit it says in the header")
    del jhgasdhgjkahsdkgdfjhsgjgjsdhkfjghjd #this causes an error and stops execution 

try:

    # Get the beam size. If there is none, use the pixel size
    beam_major =  (header["BMAJ"]*CUNIT).to(u.arcsec) #degrees beam size -> arcsec
    beam_minor =  (header["BMIN"]*CUNIT).to(u.arcsec) #if these two are different, something went wrong in the reprojection 
    Cube_Information['Beam_Position_angle'] = header['bpa']*u.deg


except:

    cdelt_x = u.Quantity(str(np.abs(header['cdelt1']))+header['cunit1'])
    cdelt_y = u.Quantity(str(np.abs(header['cdelt2']))+header['cunit2'])
    if(cdelt_x>cdelt_y):
        beam_major=cdelt_x
        beam_minor=cdelt_y
    elif(cdelt_x<cdelt_y):
        beam_major=cdelt_y
        beam_minor=cdelt_x
    elif(cdelt_x==cdelt_y):
        beam_major=cdelt_x
        beam_minor=cdelt_x



Pointing_Information["original_BMAJ"]=beam_major
Pointing_Information["original_BMIN"]=beam_minor
Pointing_Information["original_BMAJ_pc"]=beam_major.to(u.rad)*Pointing_Information['distance']
Pointing_Information["original_BMIN_pc"]=beam_minor.to(u.rad)*Pointing_Information['distance']
Pointing_Information["desired_beam_size"] = desired_beam_size #ill put it in a circular beam at this size


#this accounts for elliptical beams:            
Pointing_Information['original_pixel_scale_x'] = (abs(header["CDELT1"])*CUNIT.to(u.arcsec))/u.pix
Pointing_Information['original_pixel_scale_y'] = (abs(header["CDELT2"])*CUNIT.to(u.arcsec))/u.pix
Pointing_Information['original_spatial_scale_x'] = (abs(header["CDELT1"])*CUNIT.to(u.rad))/u.pix*Pointing_Information['distance']
Pointing_Information['original_spatial_scale_y'] = (abs(header["CDELT2"])*CUNIT.to(u.rad))/u.pix*Pointing_Information['distance']#convert to pc using the distance

average_pixel=np.sqrt((abs(header["CDELT1"])*CUNIT.to(u.arcsec))/u.pix*(abs(header["CDELT2"])*CUNIT.to(u.arcsec))/u.pix)

Pointing_Information['original_beam_oversampling_MAJ'] = beam_major/average_pixel
Pointing_Information['original_beam_oversampling_MIN'] = beam_minor/average_pixel
Pointing_Information['desired_beam_oversampling'] = ovs
#Pointing_Information['orig_FOV']= Crop_Around_Center(sc,Pointing_Information['target_image_rotation'],Pointing_Information['center'],desired_FOV,Pointing_Information['distance'])[1] #returns the current fov
Pointing_Information['desired_FOV']=desired_FOV

######

Cube_Information = Update_Cube_Information(Pointing_Information,Pointing_Information["Original_File_Name"])


done



In [4]:
#If using the coadded spliced mosaic:
if(Pointing_Information["Original_File_Name"] == "TTTT.fits"):


    files = [Pointing_Information["Original_File_Name"]]
    sc = SpectralCube.read("Spectral Cubes/"+files[0])  
    Cube_Name = files[0]+"_Header_Fix.fits"
    print(1)
    sc.allow_huge_operations=True
    #sc = sc.with_spectral_unit(u.km/u.s,velocity_convention="radio").to(u.K) # Change units from Hz to km/s
    sc = sc.with_spectral_unit(u.km/u.s,velocity_convention="radio")*u.K # Change units from Hz to km/s


    new = SpectralCube(data=sc.hdu.data,wcs =WCS(sc.header),mask=sc.mask)
    new.allow_huge_operations=True
    new=new*sc[0][0][0].unit
    print(2)
    del sc
    sc=new
    print(3)
    del new

    sc.write("Spectral Cubes/"+Cube_Name,overwrite=True)
    print(4)
    print("done")



    # Pointing_Information

    Pointing_Information = {}

    #This is the stuff that needs changing between cubes

    Pointing_Information["Original_File_Name"] = "TTTT.fits_Header_Fix.fits" #For the remade header with the reproject coadd
    #"CHIMPS2_CMZ_12COall_beammatch_253_reproject_specinterp.matched.noise_matched.fits"
    #"_cmz_CO_J3_2_Spliced_Smoothed_003-357.fits" #the name of the initial SC file.

    Pointing_Information["File_Descriptor"] = File_Descriptor
    Pointing_Information["target"] = "CMZ"#
    Pointing_Information["center"] = SkyCoord('-00d03m20.76s  ', '-00d02m46.176s', frame='galactic') #the center of the cmz
    Pointing_Information["crop_center"] = SkyCoord('00d00m00.00s  ', '-00d00m00.000s', frame='galactic') #the center of the image, used for cropping

    desired_beam_size = 3*u.pc #I add this to the PI later. choose this based on the largest common beam size between the compared observations
    distance = 8.178*10**3*u.pc
    Pointing_Information["distance"] = distance.to(u.Mpc) #in Mpc
    Pointing_Information["target_image_rotation"]=0*u.deg #this is the rotation of the specific image, not the target. (use clockwise rotation angle)
    Pointing_Information["target_inclination"]=0*u.deg
    Pointing_Information["target_velocity"]=0*u.km/u.s #the speed NGC253 is moving away from us
    Pointing_Information["vaxis"]=0 #which axis is the velocity
    Pointing_Information["desired_velocity_resolution"] = 2.5*u.km/u.s
    ovs = 5 #how much do you desire to oversample the beam by
    desired_FOV = [140,700]*u.pc# pc normal to the disk, across the disk
    Pointing_Information["Corresponding_Continuum"] = 'gc_850micron_dust.fits' # this is for the band 7, not for HCN J1-0 #The continuum image for this band
    #                                                 '4.3pc_beam_CMZ_850um_Cont_140x800pc.fits' # this is the JCMT continuum for 850microns (the CO 3-2 and HCN, HCO 4-3)


    print("done")

    gc.collect



    #This stuff will input automagically if the rest is correct

    sc = SpectralCube.read("Spectral Cubes/"+Pointing_Information["Original_File_Name"])
    header = sc.header
    print()
    try:
        freq = header["RESTFREQ"]*u.Hz# assuming the header is in Hz
        Pointing_Information['wavelength']=299792458*u.m/header["RESTFREQ"]#
        Pointing_Information['restfreq']=freq         
    except:
        freq = header["RESTFRQ"]*u.Hz#
        Pointing_Information['wavelength']=299792458*u.m/header["RESTFRQ"]#            
        Pointing_Information['restfreq']=freq 

    ######calculate teh beam size of the original image:

    if (header['CUNIT1'].find("deg")!=-1):
        CUNIT = 1*u.degree
        Pointing_Information["CUNIT"]=CUNIT
    else:
        print("The header should show CUNIT in degrees. If not, just fix this or write CUNIT = the unit it says in the header")
        del jhgasdhgjkahsdkgdfjhsgjgjsdhkfjghjd #this causes an error and stops execution 

    try:

        # Get the beam size. If there is none, use the pixel size
        beam_major =  (header["BMAJ"]*CUNIT).to(u.arcsec) #degrees beam size -> arcsec
        beam_minor =  (header["BMIN"]*CUNIT).to(u.arcsec) #if these two are different, something went wrong in the reprojection 
        Cube_Information['Beam_Position_angle'] = header['bpa']*u.deg


    except:

        cdelt_x = u.Quantity(str(np.abs(header['cdelt1']))+header['cunit1'])
        cdelt_y = u.Quantity(str(np.abs(header['cdelt2']))+header['cunit2'])
        if(cdelt_x>cdelt_y):
            beam_major=cdelt_x
            beam_minor=cdelt_y
        elif(cdelt_x<cdelt_y):
            beam_major=cdelt_y
            beam_minor=cdelt_x
        elif(cdelt_x==cdelt_y):
            beam_major=cdelt_x
            beam_minor=cdelt_x



    Pointing_Information["original_BMAJ"]=beam_major
    Pointing_Information["original_BMIN"]=beam_minor
    Pointing_Information["original_BMAJ_pc"]=beam_major.to(u.rad)*Pointing_Information['distance']
    Pointing_Information["original_BMIN_pc"]=beam_minor.to(u.rad)*Pointing_Information['distance']
    Pointing_Information["desired_beam_size"] = desired_beam_size #ill put it in a circular beam at this size


    #this accounts for elliptical beams:            
    Pointing_Information['original_pixel_scale_x'] = (abs(header["CDELT1"])*CUNIT.to(u.arcsec))/u.pix
    Pointing_Information['original_pixel_scale_y'] = (abs(header["CDELT2"])*CUNIT.to(u.arcsec))/u.pix
    Pointing_Information['original_spatial_scale_x'] = (abs(header["CDELT1"])*CUNIT.to(u.rad))/u.pix*Pointing_Information['distance']
    Pointing_Information['original_spatial_scale_y'] = (abs(header["CDELT2"])*CUNIT.to(u.rad))/u.pix*Pointing_Information['distance']#convert to pc using the distance

    average_pixel=np.sqrt((abs(header["CDELT1"])*CUNIT.to(u.arcsec))/u.pix*(abs(header["CDELT2"])*CUNIT.to(u.arcsec))/u.pix)

    Pointing_Information['original_beam_oversampling_MAJ'] = beam_major/average_pixel
    Pointing_Information['original_beam_oversampling_MIN'] = beam_minor/average_pixel
    Pointing_Information['desired_beam_oversampling'] = ovs
    #Pointing_Information['orig_FOV']= Crop_Around_Center(sc,Pointing_Information['target_image_rotation'],Pointing_Information['center'],desired_FOV,Pointing_Information['distance'])[1] #returns the current fov
    Pointing_Information['desired_FOV']=desired_FOV

    ######

    Cube_Information = Update_Cube_Information(Pointing_Information,Pointing_Information["Original_File_Name"])
print("done")

done


# 1. Reprojection

In [5]:
gc.collect()

#
# def Reproject_To_Region(Pointing_Information,Cube_Information,i_step=30,Cube_Name_Save='',Force_Origin=[False,[0,0]*u.deg,[0,0]*u.deg]):

Reproject_To_Region(Pointing_Information,Cube_Information,i_step=30,Cube_Name_Save='',Force_Origin=[True,[357,3],[-.6,.6]])

# This is the cube name that they are being saved to unless you changed it inside the reproject functions definiton

Cube_Name_Save = str(Pointing_Information["desired_beam_size"].value)+"pc_beam_"+Pointing_Information["File_Descriptor"]+str(Pointing_Information["desired_FOV"][0].value)+"x"+str(Pointing_Information["desired_FOV"][1].value)+'pc_'+'reprojected.fits'

gc.collect()

print("done")


begin step: 0 of 7
start beam convolution
0.0 0.0
0.0003973228514373017 0.0003973228514373017
convolve end

fov crop start 

Center: <SkyCoord (Galactic): (l, b) in deg
    (0., -0.)> Pixel center: 612 124 Pixel FOV: [207, 1038]
cropping cube. rotation: 0.0 deg center: <SkyCoord (Galactic): (l, b) in deg
    (0., -0.)> crop to: [140. 700.] pc
Cropped to  [140. 700.] pc
check max SC value after crop: 0.20225668342338837 SC shape: (30, 197, 856)
fov cropped

Force values of: [357, 3] [-0.6, 0.6] Giving pixel range of  1427 285
Delete LBOUND if there is one in the header, if not this will pass the error:
"Keyword 'LBOUND1' not found."
------------------------------------------------------------
Traceback (most recent call last):
  File "/tmp/ipykernel_528483/393924254.py", line 274, in Reproject_To_Region
    del reheader['LBOUND1']
  File "/home/ben/.local/lib/python3.8/site-packages/astropy/io/fits/header.py", line 239, in __delitem__
    raise KeyError(f"Keyword '{key}' not found.")
Ke

# 1a. Splice the reprojected cubes back together

In [6]:
# First, splice the partwise cubes back together

Cube_Name_Save = str(Pointing_Information["desired_beam_size"].value)+"pc_beam_"+Pointing_Information["File_Descriptor"]+str(Pointing_Information["desired_FOV"][0].value)+"x"+str(Pointing_Information["desired_FOV"][1].value)+'pc_'+'reprojected.fits'

Splice_vels(Cube_Name_Load=Cube_Name_Save)

Cube_Information = Update_Cube_Information(Pointing_Information,Cube_Name_Save)

gc.collect()

print("done")

Loaded 0_3.0pc_beam_cmz_HCN_J4_3_140.0x700.0pc_reprojected.fits
Loaded 1_3.0pc_beam_cmz_HCN_J4_3_140.0x700.0pc_reprojected.fits
Loaded 2_3.0pc_beam_cmz_HCN_J4_3_140.0x700.0pc_reprojected.fits
Loaded 3_3.0pc_beam_cmz_HCN_J4_3_140.0x700.0pc_reprojected.fits
Loaded 4_3.0pc_beam_cmz_HCN_J4_3_140.0x700.0pc_reprojected.fits
Loaded 5_3.0pc_beam_cmz_HCN_J4_3_140.0x700.0pc_reprojected.fits
Loaded 6_3.0pc_beam_cmz_HCN_J4_3_140.0x700.0pc_reprojected.fits
[Errno 2] No such file or directory: 'Spectral Cubes/7_3.0pc_beam_cmz_HCN_J4_3_140.0x700.0pc_reprojected.fits'
------------------------------------------------------------
Traceback (most recent call last):
  File "/tmp/ipykernel_528483/3881702570.py", line 12, in Splice_vels
    sc=SpectralCube.read(("Spectral Cubes/"+Cube_Name_Load_p))
  File "/home/ben/.local/lib/python3.8/site-packages/spectral_cube/io/core.py", line 118, in __call__
    return registry.read(BaseSpectralCube, filename, *args, **kwargs)
  File "/home/ben/.local/lib/python3.8/s

# 1b. Velocity reprojection

In [7]:
Cube_Name_Save = str(Pointing_Information["desired_beam_size"].value)+"pc_beam_"+Pointing_Information["File_Descriptor"]+str(Pointing_Information["desired_FOV"][0].value)+"x"+str(Pointing_Information["desired_FOV"][1].value)+'pc_'+'reprojected.fits'

Cube_Information = Update_Cube_Information(Pointing_Information,Cube_Name_Save)

# Load the spliced cube

sc = SpectralCube.read(("Spectral Cubes/"+Cube_Name_Save)) 

# Reproject the spliced Cube's veloctiy axis

Repo_Velocity(sc,Cube_Name_Save,Cube_Information)

# Here is the new cube's name

Cube_Name_Save = Cube_Name_Save[0:len(Cube_Name_Save)-5]+"_"+str(Pointing_Information["desired_velocity_resolution"].value)+'_vel_res_'+'.fits'

del sc

gc.collect()

Start velocity reprojection of 3.0pc_beam_cmz_HCN_J4_3_140.0x700.0pc_reprojected.fits
Smoothed to Gaussian Kernel of width 0.6354836718449597
Wrote reprojected cube to Spectral Cubes/3.0pc_beam_cmz_HCN_J4_3_140.0x700.0pc_reprojected_2.5_vel_res_.fits


0

# 1c. Fix repeated pixels

In [8]:
# after the reprojection, there may be duplicated data from the fact that it just fills in stuff 
# expecting new data to be there. This fixes it by checking to see if it matches other data, then removes
# the repeated slices. in other words, it may be set to reproject to 0-500 km/s through the velocity channels
# but there may only be data between 100-400, so it will fill it in with copied data channels. This will check for those.



# Update to the current cube name

Cube_Name_Save = str(Pointing_Information["desired_beam_size"].value)+"pc_beam_"+Pointing_Information["File_Descriptor"]+str(Pointing_Information["desired_FOV"][0].value)+"x"+str(Pointing_Information["desired_FOV"][1].value)+'pc_'+'reprojected.fits'
Cube_Name_Save = Cube_Name_Save[0:len(Cube_Name_Save)-5]+"_"+str(Pointing_Information["desired_velocity_resolution"].value)+'_vel_res_'+'.fits'

Cube_Information = Update_Cube_Information(Pointing_Information,Cube_Name_Save)

# Run the function to crop the nan values out of the cube

Remove_Repeated_Pixels(Cube_Name_Save)

Good data starting from channel 20 ; start has been cropped to that channel
Good data ending at channel 180 ; end has been cropped to that channel
Cropped cube saved as  Spectral Cubes/Cropped_3.0pc_beam_cmz_HCN_J4_3_140.0x700.0pc_reprojected_2.5_vel_res_.fits


# x. save cube information

In [9]:
# Now that all the data reduction is concluded, save information about the new cubes

Cube_Name_Save = str(Pointing_Information["desired_beam_size"].value)+"pc_beam_"+Pointing_Information["File_Descriptor"]+str(Pointing_Information["desired_FOV"][0].value)+"x"+str(Pointing_Information["desired_FOV"][1].value)+'pc_'+'reprojected.fits'
Cube_Name_Save = "Cropped_"+Cube_Name_Save[0:len(Cube_Name_Save)-5]+"_"+str(Pointing_Information["desired_velocity_resolution"].value)+'_vel_res_'+'.fits'

Cube_Information = Update_Cube_Information(Pointing_Information,Cube_Name_Save)

import pickle 

with open("Spectral Cubes/Cube Information/"+File_Descriptor+"_Cube_Information", 'wb') as file:
    pickle.dump(Cube_Information, file)

with open("Spectral Cubes/Cube Information/"+File_Descriptor+"_Pointing_Information", 'wb') as file:
    pickle.dump(Pointing_Information, file)

print(Cube_Information)

{'Original_File_Name': 'hcn4_3_KT18_Tmb.fits', 'File_Descriptor': 'cmz_HCN_J4_3_', 'target': 'CMZ', 'center': <SkyCoord (Galactic): (l, b) in deg
    (359.94423333, -0.04616)>, 'crop_center': <SkyCoord (Galactic): (l, b) in deg
    (0., -0.)>, 'distance': <Quantity 0.008178 Mpc>, 'target_image_rotation': <Quantity 0. deg>, 'target_inclination': <Quantity 0. deg>, 'target_velocity': <Quantity 0. km / s>, 'vaxis': 0, 'desired_velocity_resolution': <Quantity 2.5 km / s>, 'Corresponding_Continuum': 'gc_850micron_dust.fits', 'wavelength': <Quantity 0.00084566 m>, 'restfreq': <Quantity 3.545055e+11 Hz>, 'CUNIT': <Quantity 1. deg>, 'original_BMAJ': <Quantity 0.00236111 deg>, 'original_BMIN': <Quantity 0.00236111 deg>, 'original_BMAJ_pc': <Quantity 3.37008518e-07 Mpc rad>, 'original_BMIN_pc': <Quantity 3.37008518e-07 Mpc rad>, 'desired_beam_size': <Quantity 3. pc>, 'original_pixel_scale_x': <Quantity 8.4999996 arcsec / pix>, 'original_pixel_scale_y': <Quantity 8.4999996 arcsec / pix>, 'origina

# xb. Noise matching

# Return an input cube matched to the given nosie

Cube_Name_Save = str(Pointing_Information["desired_beam_size"].value)+"pc_beam_"+Pointing_Information["File_Descriptor"]+str(Pointing_Information["desired_FOV"][0].value)+"x"+str(Pointing_Information["desired_FOV"][1].value)+'pc_'+'reprojected.fits'
Cube_Name_Save = "Cropped_"+Cube_Name_Save[0:len(Cube_Name_Save)-5]+"_"+str(Pointing_Information["desired_velocity_resolution"].value)+'_vel_res_'+'.fits'

Q  = SpectralCube.read("Spectral Cubes/"+Cube_Name_Save)

NM_Cube = Noise_matching(Input_Cube=Q,m=(.115*u.K))
    
    
Cube_Name_Save = "NM_"+Cube_Name_Save

NM_Cube.write("Spectral Cubes/"+Cube_Name_Save)
    


Cube_Information = Update_Cube_Information(Pointing_Information,Cube_Name_Save)

import pickle 

with open("Spectral Cubes/Cube Information/"+File_Descriptor+"_Cube_Information", 'wb') as file:
    pickle.dump(Cube_Information, file)

with open("Spectral Cubes/Cube Information/"+File_Descriptor+"_Pointing_Information", 'wb') as file:
    pickle.dump(Pointing_Information, file)

print(Cube_Information)